The World Health Organization provides daily updated (by 10 am Geneva time) csv files on countries' confirmed cases of COVID-19. This notebook will lay out some visualizations / analysis, which can be run daily from your computer in order to watch the numbers yourself.

In [23]:
# Import analytical libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import date
import plotly.express as px

In [14]:
# Print the date so we can see the last time this data was updated. 
today = date.today()

print(f'Data as of {today}')

data = pd.read_csv('https://covid.ourworldindata.org/data/full_data.csv')
data.head()

Data as of 2020-03-17


,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2020-02-25,Afghanistan,NaN,NaN,1.0,NaN
1,2020-02-26,Afghanistan,0.0,NaN,1.0,NaN
2,2020-02-27,Afghanistan,0.0,NaN,1.0,NaN
3,2020-02-28,Afghanistan,0.0,NaN,1.0,NaN
4,2020-02-29,Afghanistan,0.0,NaN,1.0,NaN


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 6 columns):
date            2706 non-null object
location        2706 non-null object
new_cases       2553 non-null float64
new_deaths      492 non-null float64
total_cases     2705 non-null float64
total_deaths    538 non-null float64
dtypes: float64(4), object(2)
memory usage: 127.0+ KB


In [16]:
# Isolate the World location for today's stats
data[(data['date'] == data['date'].max()) & (data['location'] == 'World')]

,date,location,new_cases,new_deaths,total_cases,total_deaths
2705,2020-03-16,World,13968.0,855.0,167502.0,6603.0


In [17]:
# Create a df slice for today's data

latest_data = data[data['date'] == data['date'].max()]

In [18]:
# Create a slice of the df for non-country specific data
world_index = latest_data[(latest_data['location'] == 'World') | (latest_data['location'] == 'International')].index

# Drop the world_index slice of the dataframe in a new variable called geo_df
geo_df = latest_data.drop(world_index)

In [19]:
# Isolate your country
my_country = 'Israel'

# Check your countries stats
geo_df[geo_df['location']== my_country]

,date,location,new_cases,new_deaths,total_cases,total_deaths
1186,2020-03-16,Israel,22.0,NaN,200.0,NaN


In [20]:
# Import libraries for choropleth maps 

import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True) 

In [21]:
# Input the relevant information, check go.Figure documentation for help

geo_data = dict(type='choropleth',
            colorscale = 'plasma', # Optional colors for readability
            locations = geo_df['location'], # Remove the world data row
            z = geo_df['new_cases'], # Numbers to plot
            locationmode = 'country names', # Specify how locations should be read 
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 4)), # Colors for Readability
            colorbar = {'title':"New Cases per Country"} # Colorbar title
            ) 

layout = dict(title = f'New Cases of COVID-19 as of {today}', # Map title
              geo = dict(scope='world') # Map scope
             )

# Assign the variables to the go.Figure method
choromap = go.Figure(data = [geo_data],layout = layout)

In [22]:
# Display the graph
iplot(choromap)

In [35]:
# Select the 10 countries with the most total cases

most_cases = geo_df.groupby('location').max().sort_values('total_cases', ascending=False).head(10)

In [42]:
# Plot the top 10 countries 

fig = px.bar(most_cases.reset_index(), x='location', y='total_cases',
             color='new_cases')
fig.show()

In [79]:
# Plot the new cases per day
# Yesterday, March 16 had the highest increse in cases since Feb 17

fig= px.line(data[data['location']=='World'], x='date', y='new_cases')
fig.show()